In [1]:
#though when we convert to onnx then model size reduce but now we will do quantisation to
#make it run on edge devices

#before stats
# tf, gpu = 0.15s
# tf, cpu = 0.8s
# tf_size = 1000MB

# onnx, cpu = 0.5s
# onnx, gpu = 0.025s
# onnx_size = 328MB

# onnx_quantized, cpu = 0.4s
# onnx_quantized, gpu = 0.3s
# onnx_quantized_size = 83MB

In [3]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.applications.vgg16 import VGG16

from keras.applications import VGG16
conv_base = VGG16(weights='imagenet',
    include_top = False,
    input_shape=(224,224,3))

conv_base.trainable = True

set_trainable = False

for layer in conv_base.layers:
  #if layer.name == 'conv2d_81':
  if layer.name=='input_1':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False
initializer = tensorflow.keras.initializers.HeNormal(seed=42)
model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(128,activation='relu',kernel_initializer=initializer))
model.add(Dense(128,activation='relu',kernel_initializer=initializer))
model.add(Dense(2,activation='softmax',kernel_initializer=initializer))
model.compile(
    optimizer=tensorflow.keras.optimizers.SGD(learning_rate=0.001,momentum=0.9,nesterov=True,weight_decay=0.0005),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
  )
import numpy as np
X=np.random.normal(size=(10,224,224,3))
y=np.array([1,0,1,1,0,0,1,1,0,0])

history=model.fit(
    X, y,
    epochs=1,
    batch_size=2
)


print(history.history['accuracy'])

5/5 [==============================] - 29s 5s/step - loss: 1.3881 - accuracy: 0.5000
[0.5]


In [4]:
! pip install -U tf2onnx
! pip install onnxruntime

import tensorflow as tf
import tf2onnx
import onnxruntime as rt
spec = (tf.TensorSpec(
    (None,
     224, #CONFIGURATION[ "IM_SIZE"],
     224, #CONFIGURATION["IM_SIZE"]
     3), tf.float32, name="input"),)

output_path = "vgg16_keras.onnx"

model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13, output_path=output_path)
#opset tell about the version
output_names = [n.name for n in model_proto.graph.output]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.6 MB/s eta 0:00:00


In [5]:
#qunatisation

In [6]:
import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType

In [7]:
model_fp32 = '/content/vgg16_keras.onnx'
model_quant = '/content/vgg16_keras_quantized.onnx'

quantized_model = quantize_dynamic(model_fp32, model_quant, weight_type = QuantType.QUInt8)


In [8]:
#size reduced to 17mb only

In [9]:
#inference


import onnxruntime as rt

output_path = "/content/vgg16_keras_quantized.onnx"
providers = ['CPUExecutionProvider']

m = rt.InferenceSession(output_path, providers=providers)

im=np.random.normal(size=(3,224,224,3))
im=np.float32(im)

m.run(output_names, {"input": im})

[array([[0.71810126, 0.28189877],
        [0.7249614 , 0.27503857],
        [0.7163054 , 0.28369457]], dtype=float32)]

In [10]:
import time

t1=time.time()
m.run(output_names, {"input": im})
print(time.time()-t1)

#onnx model takes lesser time

1.7529897689819336


In [11]:
#accuracy drop due to quantisation

# def accuracy(model):
#   total, acc = 0,0
#   for im, label in validation_dataset:
#     onnx_pred = model.run(output_names, {"input": np.array(im)})

#     if(int(np.argmax(onnx_pred, axis = -1)[0][0]) == int(np.argmax(label, axis = -1)[0])):
#       acc += 1

#     total += 1
#   return acc/total

# providers=['CUDAExecutionProvider']
# m = rt.InferenceSession("/content/drive/MyDrive/Bang/vit_keras.onnx", providers=providers)
# m_q = rt.InferenceSession("/content/vit_quantized.onnx", providers=providers)
# print(accuracy(m_q))
# print(accuracy(m))

# 0.9051799824407375
# 0.9051799824407375

In [12]:
# QUAntisation Aware training

In [13]:
model.summary() #here we have vgg16 as one model inside our model so we need to have one model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 128)               3211392   
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 2)                 258       
                                                                 
Total params: 17942850 (68.45 MB)
Trainable params: 3228162 (12.31 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [4]:
x=Flatten()(conv_base.output)
x=Dense(128,activation='relu',kernel_initializer=initializer)(x)
x=Dense(128,activation='relu',kernel_initializer=initializer)(x)
x=Dense(2,activation='softmax',kernel_initializer=initializer)(x)

model_new =keras.Model(inputs=conv_base.input, outputs= x)

model_new.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [6]:
model_new.compile(
    optimizer=tensorflow.keras.optimizers.SGD(learning_rate=0.001,momentum=0.9,nesterov=True,weight_decay=0.0005),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
  )
import numpy as np
X=np.random.normal(size=(10,224,224,3))
y=np.array([1,0,1,1,0,0,1,1,0,0])

history=model_new.fit(
    X, y,
    epochs=1,
    batch_size=2
)
model_new.save('/content/simple_vgg16/')

5/5 [==============================] - 28s 5s/step - loss: 1.0484 - accuracy: 0.4000


In [15]:
!pip install -q tensorflow-model-optimization

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 3.9 MB/s eta 0:00:00


In [29]:
#we can select some layers that we want to quantised
#layers like batchnorm, type cannot be normalised with this tf quantisation

import tensorflow_model_optimization as tfmot
input=keras.layers.Input(shape=(224,224,3), name='input')
conv=keras.layers.Conv2D(32, kernel_size=3, activation='relu', name='conv_layer')(input)
x=Flatten(name='flatten')(conv)
x=Dense(128,activation='relu',kernel_initializer=initializer, name='l1')(x)
x=Dense(128,activation='relu',kernel_initializer=initializer, name='l2')(x)
x=Dense(2,activation='softmax',kernel_initializer=initializer, name='l3')(x)

model_noob =keras.Model(inputs=input, outputs= x)

In [30]:
model_noob.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 224, 224, 3)]     0         
                                                                 
 conv_layer (Conv2D)         (None, 222, 222, 32)      896       
                                                                 
 flatten (Flatten)           (None, 1577088)           0         
                                                                 
 l1 (Dense)                  (None, 128)               201867392 
                                                                 
 l2 (Dense)                  (None, 128)               16512     
                                                                 
 l3 (Dense)                  (None, 2)                 258       
                                                                 
Total params: 201885058 (770.13 MB)
Trainable params: 20188

In [36]:
# quant_aware = tfmot.quantization.keras.quantize_model(model_noob)
#this doesnot work

In [28]:
#we will select layer by layer which we want to be quantise aware
def apply_quantization_to_conv(layer):
  if "conv" in layer.name:
    return tfmot.quantization.keras.quantize_annotate_layer(layer) #this for maarking which layer to make Q.A.
  return layer

In [33]:
quant_aware_eff = tf.keras.models.clone_model(
    model_new, clone_function=apply_quantization_to_conv
) #this is for cloning model so that apply the function on convo layers to make them Q.A.

In [35]:
quant_aware_eff.summary() #we get something different can seee

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 quantize_annotate_22 (Quan  (None, 224, 224, 64)      1792      
 tizeAnnotate)                                                   
                                                                 
 quantize_annotate_23 (Quan  (None, 224, 224, 64)      36928     
 tizeAnnotate)                                                   
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 quantize_annotate_24 (Quan  (None, 112, 112, 128)     73856     
 tizeAnnotate)                                                   
                                                           

In [37]:
#now we can compile and train it as regular model

quant_aware_eff.compile(
    optimizer=tensorflow.keras.optimizers.SGD(learning_rate=0.001,momentum=0.9,nesterov=True,weight_decay=0.0005),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
  )
import numpy as np
X=np.random.normal(size=(10,224,224,3))
y=np.array([1,0,1,1,0,0,1,1,0,0])

history=quant_aware_eff.fit(
    X, y,
    epochs=5,
    batch_size=2
)

Epoch 1/5
5/5 [==============================] - 7s 1s/step - loss: 1.5670 - accuracy: 0.4000
Epoch 2/5
5/5 [==============================] - 7s 1s/step - loss: 1.3473 - accuracy: 0.3000
Epoch 3/5
5/5 [==============================] - 5s 1s/step - loss: 1.4986 - accuracy: 0.3000
Epoch 4/5
5/5 [==============================] - 7s 1s/step - loss: 0.9739 - accuracy: 0.3000
Epoch 5/5
5/5 [==============================] - 5s 1s/step - loss: 1.0386 - accuracy: 0.5000


In [ ]:
#post training quantisation

#doing quantisation on already trained model

In [ ]:
import tensorflow as tf
from tensorflow import keras
model_new=keras.models.load_model('/content/simple_vgg16')
model_new.summary()

In [21]:
import numpy as np
X=np.random.normal(size=(10,224,224,3))
X=np.float32(X)
y=np.array([1,0,1,1,0,0,1,1,0,0])

In [22]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_new)

converter.optimizations = [tf.lite.Optimize.DEFAULT] #this we can refer to docs and these are attributes that we can
#define that tell for which we want to optimize speed , memory etc

converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8 #in this we define inference input and output type

#for defining representation of data we make generator that give input values as static quant need some inputs
#to find s and z
def representative_data_gen():
  for input_value,j in zip(X,y): #training_dataset.take(20):
    yield [input_value.reshape(1,224,224,3)]

converter.representative_dataset = representative_data_gen #if want to use dynamic quantisatn
#then dont specify data geenrator here

In [23]:
tflite_model = converter.convert()

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [24]:
#now we save conversion in tflite format

import pathlib

tflite_models_dir = pathlib.Path("/content/quantized_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

tflite_model_file = tflite_models_dir/"eff_model.tflite"
tflite_model_file.write_bytes(tflite_model)

18069424

In [25]:
#we get 17 mb model from 136mb model

In [26]:
#tflite runtime
!pip3 install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

#this is used to run tflite model on any small devices

Looking in indexes: https://pypi.org/simple, https://google-coral.github.io/py-repo/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 10.9 MB/s eta 0:00:00


In [27]:
import tflite_runtime as tflite
import numpy as np
import cv2

#we dont need tensorflow in small devices only take small libraries to read images and all and use
#tflite runtime to do inference

In [40]:
# test_image = cv2.imread("/content/dataset/Emotions Dataset/Emotions Dataset/train/happy/148266.jpg")
# test_image = cv2.resize(test_image, (CONFIGURATION["IM_SIZE"] ,CONFIGURATION["IM_SIZE"]))
# test_image = np.expand_dims(test_image, axis = 0)
#print(CONFIGURATION['CLASS_NAMES'][tf.argmax(pretrained_model(im), axis = -1).numpy()[0]])

test_image=np.zeros((1,224,224,3), dtype='float32')

In [44]:
import tflite_runtime.interpreter as tflite
interpreter = tflite.Interpreter(model_path="/content/quantized_models/eff_model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]
print(input_details['dtype'])
print(input_details['index'])

# test_image = test_image.numpy().astype(input_details["dtype"])
test_image = test_image.astype(input_details["dtype"]) #converting float32 to uint8

interpreter.set_tensor(input_details["index"], test_image) #set tensor
interpreter.invoke() #then do inference

output = interpreter.get_tensor(output_details["index"])[0] #get tensor at the level of output

<class 'numpy.uint8'>
0


In [39]:
np.argmax(output)

1

In [48]:
#now measure accuracy of tflte quantised model
# def accuracy(model_path):
#   total, correct = 0,0
#   interpreter = tf.lite.Interpreter(model_path=model_path) #here for checking val acc on system we are using lite from tf
#   interpreter.allocate_tensors()

#   input_details = interpreter.get_input_details()[0]
#   output_details = interpreter.get_output_details()[0]


#   for im, label in validation_dataset:

#     test_image = im.numpy().astype(input_details["dtype"])

#     interpreter.set_tensor(input_details["index"], test_image)
#     interpreter.invoke()
#     output = interpreter.get_tensor(output_details["index"])[0]

#     if(int(np.argmax(output)) == int(np.argmax(label, axis = -1)[0])):
#       correct += 1

#     total += 1
#   return correct/total

# accuracy("/content/drive/MyDrive/Bang/eff_model.tflite")